In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [2]:
# Hyperparameters
batch_size = 64
learning_rate = 1e-3
num_epochs = 20

# Transform: Normalize and Flatten Images
transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Lambda(lambda x: x.view(-1))  # Flatten the image
])

# Load CIFAR10 Dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

Files already downloaded and verified
Files already downloaded and verified
cuda


<a id="3"></a> <br>
### Logistic Regression

In [5]:
# Create Logistic Regression Model
class LogisticRegressionModel(nn.Module):
    def __init__(self):
        super(LogisticRegressionModel, self).__init__()

        self.linear = nn.Linear(32 * 32 * 3, 10)

    def forward(self, x):
        out = self.linear(x.view(-1, 32 * 32 * 3))
        return out


model = LogisticRegressionModel().to(device)

error = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [12]:
# Traning the Model
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for (images, labels) in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = error(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    running_loss += loss.item()
        
    # Calculate Accuracy         
    correct = 0
    total = 0
    # Predict test dataset
    for images, labels in test_loader: 
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        predicted = torch.max(outputs.data, 1)[1]
        total += len(labels)
        correct += (predicted == labels).sum()
    
    accuracy = 100 * correct / float(total)
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch}/{num_epochs}], Loss: {avg_loss:.4f}, Acc: {accuracy:.4f}%")

Epoch [0/20], Loss: 0.0027, Acc: 29.5100%
Epoch [1/20], Loss: 0.0026, Acc: 32.1100%
Epoch [2/20], Loss: 0.0028, Acc: 33.3300%
Epoch [3/20], Loss: 0.0024, Acc: 34.7900%
Epoch [4/20], Loss: 0.0024, Acc: 35.3800%
Epoch [5/20], Loss: 0.0022, Acc: 35.9000%
Epoch [6/20], Loss: 0.0027, Acc: 36.0600%
Epoch [7/20], Loss: 0.0025, Acc: 36.5900%
Epoch [8/20], Loss: 0.0023, Acc: 36.2700%
Epoch [9/20], Loss: 0.0021, Acc: 36.8800%
Epoch [10/20], Loss: 0.0024, Acc: 36.4600%
Epoch [11/20], Loss: 0.0019, Acc: 37.5400%
Epoch [12/20], Loss: 0.0021, Acc: 37.5100%
Epoch [13/20], Loss: 0.0031, Acc: 36.9700%
Epoch [14/20], Loss: 0.0023, Acc: 37.8300%
Epoch [15/20], Loss: 0.0022, Acc: 37.9300%
Epoch [16/20], Loss: 0.0023, Acc: 37.8600%
Epoch [17/20], Loss: 0.0018, Acc: 38.3800%
Epoch [18/20], Loss: 0.0026, Acc: 38.5000%
Epoch [19/20], Loss: 0.0028, Acc: 38.0500%


In [9]:
# visualization
# plt.plot(iteration_list,loss_list)
# plt.xlabel("Number of iteration")
# plt.ylabel("Loss")
# plt.title("Logistic Regression: Loss vs Number of iteration")
# plt.show()

<a id="4"></a> <br>
### Artificial Neural Network (ANN)

In [20]:
# Create ANN Model
class ANNModel(nn.Module):
    def __init__(self):
        super(ANNModel, self).__init__()

        self.fc1 = nn.Linear(32 * 32 * 3, 128) 
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 32)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(32, 10)  
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.relu3(out)
        out = self.fc4(out)
        return out


model = ANNModel().to(device)
error = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-2)

In [21]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

# Example
total_params = count_parameters(model)
print(f"Total Parameters: {total_params}")

Total Parameters: 404010


In [22]:
num_epochs = 100

# Traning the Model
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for (images, labels) in train_loader:
        images = images.view(-1, 3 * 32 * 32).to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = error(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    running_loss += loss.item()
        
    # Calculate Accuracy         
    correct = 0
    total = 0
    # Predict test dataset
    for images, labels in test_loader: 
        images = images.view(-1, 3 * 32 * 32).to(device)
        labels = labels.to(device)

        outputs = model(images)
        predicted = torch.max(outputs.data, 1)[1]
        total += len(labels)
        correct += (predicted == labels).sum()
    
    accuracy = 100 * correct / float(total)
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch}/{num_epochs}], Loss: {avg_loss:.4f}, Acc: {accuracy:.4f}%")

Epoch [0/100], Loss: 0.0023, Acc: 34.7500%
Epoch [1/100], Loss: 0.0022, Acc: 38.5400%
Epoch [2/100], Loss: 0.0018, Acc: 40.0700%
Epoch [3/100], Loss: 0.0021, Acc: 41.9500%
Epoch [4/100], Loss: 0.0022, Acc: 43.2900%
Epoch [5/100], Loss: 0.0016, Acc: 45.0900%
Epoch [6/100], Loss: 0.0019, Acc: 46.3300%
Epoch [7/100], Loss: 0.0015, Acc: 45.0600%
Epoch [8/100], Loss: 0.0019, Acc: 46.2800%
Epoch [9/100], Loss: 0.0015, Acc: 47.4700%
Epoch [10/100], Loss: 0.0018, Acc: 47.0300%
Epoch [11/100], Loss: 0.0015, Acc: 47.8100%
Epoch [12/100], Loss: 0.0017, Acc: 47.3800%
Epoch [13/100], Loss: 0.0019, Acc: 47.5900%
Epoch [14/100], Loss: 0.0017, Acc: 49.0800%
Epoch [15/100], Loss: 0.0012, Acc: 49.4200%
Epoch [16/100], Loss: 0.0016, Acc: 48.8600%
Epoch [17/100], Loss: 0.0020, Acc: 48.5900%
Epoch [18/100], Loss: 0.0015, Acc: 49.3100%
Epoch [19/100], Loss: 0.0019, Acc: 48.5800%
Epoch [20/100], Loss: 0.0012, Acc: 50.0300%
Epoch [21/100], Loss: 0.0016, Acc: 51.1000%
Epoch [22/100], Loss: 0.0020, Acc: 50.5200

In [18]:
# # visualization loss 
# plt.plot(iteration_list,loss_list)
# plt.xlabel("Number of iteration")
# plt.ylabel("Loss")
# plt.title("ANN: Loss vs Number of iteration")
# plt.show()

# # visualization accuracy 
# plt.plot(iteration_list,accuracy_list,color = "red")
# plt.xlabel("Number of iteration")
# plt.ylabel("Accuracy")
# plt.title("ANN: Accuracy vs Number of iteration")
# plt.show()

<a id="5"></a> <br>
### Convolutional Neural Network (CNN)

In [18]:
# Create CNN Model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolutional layers with BatchNorm
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)  # BatchNorm for 32 feature maps
        
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)  # BatchNorm for 64 feature maps
        
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)  # BatchNorm for 128 feature maps
        
        self.pool = nn.MaxPool2d(2, 2)  # Downsampling
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 4 * 4, 256)
        self.bn_fc1 = nn.BatchNorm1d(256)  # BatchNorm for FC layer
        
        self.fc2 = nn.Linear(256, 10)  # Output layer for 10 classes

    def forward(self, x):
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))  # Conv -> BatchNorm -> ReLU -> Pool
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool(torch.relu(self.bn3(self.conv3(x))))
        
        x = torch.flatten(x, 1)  # Flatten for FC layer
        x = torch.relu(self.bn_fc1(self.fc1(x)))  # FC -> BatchNorm -> ReLU
        x = self.fc2(x)  # No activation for final layer (handled by loss function)
        
        return x


model = CNNModel().to(device)
error = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [19]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

# Example
total_params = count_parameters(model)
print(f"Total Parameters: {total_params}")

Total Parameters: 621322


In [43]:
num_epochs = 25

# Traning the Model
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for (images, labels) in train_loader:
        images = images.to(device).view(-1, 3, 32, 32)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = error(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    running_loss += loss.item()
        
    # Calculate Accuracy         
    correct = 0
    total = 0
    # Predict test dataset
    for images, labels in test_loader: 
        images = images.to(device).view(-1, 3, 32, 32)
        labels = labels.to(device)

        outputs = model(images)
        predicted = torch.max(outputs.data, 1)[1]
        total += len(labels)
        correct += (predicted == labels).sum()
    
    accuracy = 100 * correct / float(total)
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch}/{num_epochs}], Loss: {avg_loss:.4f}, Acc: {accuracy:.4f}%")

Epoch [0/50], Loss: 0.0010, Acc: 69.7400%
Epoch [1/50], Loss: 0.0011, Acc: 74.3500%
Epoch [2/50], Loss: 0.0008, Acc: 76.2300%
Epoch [3/50], Loss: 0.0005, Acc: 77.0800%
Epoch [4/50], Loss: 0.0008, Acc: 77.9300%
Epoch [5/50], Loss: 0.0004, Acc: 77.9400%
Epoch [6/50], Loss: 0.0003, Acc: 77.6800%
Epoch [7/50], Loss: 0.0004, Acc: 77.4900%
Epoch [8/50], Loss: 0.0013, Acc: 77.6100%
Epoch [9/50], Loss: 0.0005, Acc: 78.1000%
Epoch [10/50], Loss: 0.0000, Acc: 77.9900%
Epoch [11/50], Loss: 0.0006, Acc: 76.9700%
Epoch [12/50], Loss: 0.0000, Acc: 77.6500%
Epoch [13/50], Loss: 0.0002, Acc: 77.6400%
Epoch [14/50], Loss: 0.0007, Acc: 77.9900%
Epoch [15/50], Loss: 0.0011, Acc: 77.6900%
Epoch [16/50], Loss: 0.0001, Acc: 78.0000%
Epoch [17/50], Loss: 0.0001, Acc: 77.8400%
Epoch [18/50], Loss: 0.0001, Acc: 77.3700%
Epoch [19/50], Loss: 0.0000, Acc: 77.9000%
Epoch [20/50], Loss: 0.0000, Acc: 78.1700%
Epoch [21/50], Loss: 0.0003, Acc: 77.2900%
Epoch [22/50], Loss: 0.0002, Acc: 78.2500%
Epoch [23/50], Loss: 

In [29]:
# # visualization loss 
# plt.plot(iteration_list,loss_list)
# plt.xlabel("Number of iteration")
# plt.ylabel("Loss")
# plt.title("CNN: Loss vs Number of iteration")
# plt.show()

# # visualization accuracy 
# plt.plot(iteration_list,accuracy_list,color = "red")
# plt.xlabel("Number of iteration")
# plt.ylabel("Accuracy")
# plt.title("CNN: Accuracy vs Number of iteration")
# plt.show()